In [1]:
import ee
ee.Initialize ()

In [2]:
from iteru import *
from IPython.display import display
import ipywidgets
import datetime

In [3]:
Map = Map()
Map

Map(center=[27, 31], controls=(AttributionControl(options=['position', 'prefix'], position='bottomright'), Sea…

In [100]:
aoi = Map.aoi

In [101]:
start_year = 2020
start_month = 6
start_day = 1
end_year = 2022
end_month = 2
end_day = 23
time_delta = 20
water_threshold = -16

In [102]:
start_date = datetime.date(start_year,start_month,start_day)
end_date = datetime.date(end_year,end_month,end_day)
ee_start_date = f'{start_year}-{start_month}-{start_day}'
ee_end_date = f'{end_year}-{end_month}-{end_day}'

In [103]:
dates_list = get_dates_list(start_date ,end_date, time_delta)

In [104]:
SAR = ee.ImageCollection('COPERNICUS/S1_GRD')\
    .filter(ee.Filter.eq('instrumentMode', 'IW'))\
    .filter(ee.Filter.eq('orbitProperties_pass', 'DESCENDING'))\
    .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH'))\
    .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))\
    .filter(ee.Filter.eq('resolution_meters',10))\
    .filterBounds(aoi)\
    .filterDate(ee_start_date,ee_end_date)\
    .select(['VV','VH'])

In [105]:
SAR = SAR.map(addRatioBand)

In [106]:
def filterSpeckles(img):
    
    VV_smooth = img.select('VV').focal_median(100,'circle','meters').rename('VV_Filtered')
    
    return img.addBands(VV_smooth)

In [107]:
SAR = SAR.map(filterSpeckles)

In [108]:
def water_classify(img):
    vv  = img.select('VV_Filtered')
    #vv  = img.select('VH')
    water = vv.lt(water_threshold).rename('Water')
    water = water.updateMask(water)
 
    return img.addBands([water])

In [109]:
SAR = SAR.map(water_classify)

In [110]:
def col_vis(img):
    
    img_rgb = img.visualize(**{'min':[-25,-25,0],'max':[0,0,2],'bands':['VV','VH','VV/VH']})
    
    water_vis = img.select('Water').visualize(**{'min': 0.5,'max': 1,'palette': ['00FFFF', '0000FF']})
                  
    mosaic = ee.ImageCollection([img_rgb,water_vis]).mosaic()
 
    return mosaic.copyProperties(img, img.propertyNames())


In [111]:
SAR = SAR.map(col_vis)

In [112]:
# Map.add_layer_widgets(SAR,{'bands':'VV_Filtered','min':-25,'max':5},'TEST_SAR')

In [113]:
#Map.add_layer_widgets(SAR,{'bands':'Water','palette': ['00FFFF', '0000FF']},'TEST_SAR')

In [114]:
#Map.add_layer_widgets(SAR,{'min':[-25,-25,0],'max':[0,0,2],'bands':['VV','VH','VV/VH']},'SAR')

In [115]:
def get_images(day_date):
    
    start_day = ee.Date(day_date)
    end_day = start_day.advance(time_delta,'day')
    
    return SAR.filterDate(start_day,end_day).reduce(ee.Reducer.median())\
          .set({'start_day':start_day.format('YYYY-MM-dd')})\
          .set({'end_day':end_day.format('YYYY-MM-dd')})

In [116]:
images = ee.List(dates_list).map(get_images)

In [117]:
collection = ee.ImageCollection.fromImages(images)

In [118]:
videoArgs = {
  'dimensions': 900 ,
  'region': aoi,
  'framesPerSecond':2,
  'crs': 'EPSG:3857',
#   'min':[-25,-25,0],
#   'max':[0,0,5],
#   'bands':['VV_median','VH_median','VV/VH_median']
  #'bands':['Water_median'],
  #'palette': ['00FFFF', '0000FF']
  'bands':['vis-red_median','vis-green_median','vis-blue_median'],
   'min':0,
   'max':255
     }

In [119]:
url = collection.getVideoThumbURL(videoArgs)

In [120]:
out_gif = get_gif(url)

In [123]:
add_text_to_gif(out_gif,dates_list, dates_font_size = 25, dates_font_color = 'black', framesPerSecond = 3 )

In [124]:
display_gif(out_gif)

Output()